In [33]:
import json
import nltk
import re
import numpy as np
import tensorflow as tf
from prepro import convert_idx, save

In [34]:
with open('/home/soham/data/squad/sample1k-HCVerifyAll.json', 'r') as f:
    foo = json.load(f)

In [35]:
meh = []

In [36]:
foo['version']

'1.1'

In [37]:
i = 0; j = 0
original_paragraphs = {}
adversarial_paragraphs = {}
titles = set()

def strip_id(id_):
    return id_.split('-')[0]

for datum in foo['data']:
    titles.add(datum['title'])
    
    for paragraph in datum['paragraphs']:
        context = paragraph['context']
        
        for qa in paragraph['qas']:
            answers = qa['answers']
            question = qa['question']
            id_ = qa['id']
            
            if strip_id(id_) == id_:
                assert id_ not in original_paragraphs
                original_paragraphs[id_] = (id_, datum['title'], context, question, answers)
                i += 1
            else:
                if strip_id(id_) not in adversarial_paragraphs:
                    adversarial_paragraphs[strip_id(id_)] = []
                
                #### YOU NEED TO UNCOMMENT THIS!! #####
                adversarial_paragraphs[strip_id(id_)] += [(context, question, answers)]
                j += 1

print(i, j)

1000 2560


In [38]:
len([a for a in map(lambda x: x[0], original_paragraphs.values())])

1000

In [39]:
len(set([a for a in map(lambda x: x[0], original_paragraphs.values())]))

1000

In [40]:
np.random.seed(1337)
train_titles = set(np.random.choice(sorted(list(titles)), size=int(len(titles)*0.7), replace=False))
test_titles = titles - train_titles

In [41]:
print(', '.join(sorted(list(train_titles))))
print()
print(', '.join(sorted(list(test_titles))))

1973_oil_crisis, Amazon_rainforest, American_Broadcasting_Company, Apollo_program, Black_Death, Chloroplast, Computational_complexity_theory, Doctor_Who, Economic_inequality, European_Union_law, Force, French_and_Indian_War, Fresno,_California, Genghis_Khan, Geology, Huguenot, Imperialism, Intergovernmental_Panel_on_Climate_Change, Newcastle_upon_Tyne, Normans, Oxygen, Packet_switching, Pharmacy, Prime_number, Private_school, Rhine, Scottish_Parliament, Sky_(United_Kingdom), Southern_California, United_Methodist_Church, University_of_Chicago, Victoria_and_Albert_Museum, Warsaw

Civil_disobedience, Construction, Ctenophora, Harvard_University, Immune_system, Islamism, Jacksonville,_Florida, Kenya, Martin_Luther, Nikola_Tesla, Steam_engine, Super_Bowl_50, Teacher, Victoria_(Australia), Yuan_dynasty


3560 total:  

* 2560 mutated
* 1000 original

In [42]:
np.random.seed(1337)

In [43]:
train_pairs = []
test_pairs = []

for key, (id_, title, orig_para, orig_ques, orig_ans) in original_paragraphs.items():
     #id_, title, orig_para, orig_ques, orig_answer = id_title_para_ques_ans
    
    if title in train_titles:
        train_pairs += [(id_, orig_para, orig_ques, orig_ans, None, 'orig', title)]
    else:
        assert title in test_titles
        test_pairs += [(id_, orig_para, orig_ques, orig_ans, None, 'orig', title)]
    
    if key in adversarial_paragraphs:
        for j, (para, ques, ans) in enumerate(adversarial_paragraphs[key]):
            injection = para[len(orig_para)+1:]
            
            orig_sents = nltk.sent_tokenize(orig_para)
            
            i = np.random.randint(0, len(orig_sents) + 1)
            adv_para = ' '.join(orig_sents[:i] + [injection] + orig_sents[i:])
            
            if i == 0:
                start = 0
            else:
                start = len(' '.join(orig_sents[:i])) + 1
            
            end = start + len(injection)
            die_counter = 0
            
            for k in range(len(ans)):
                if start <= ans[k]['answer_start']:
                    new_start = ans[k]['answer_start'] + len(injection) - 1
                    pass #ans[j]['answer_start'] += len(injection)
                else:
                    new_start = ans[k]['answer_start']
                
                die_counter = 0
                
                while not adv_para[new_start:].startswith(ans[k]['text']):
                    new_start += 1
                    die_counter += 1
                    
                    if die_counter == 9999:
                        break
                
                if die_counter == 9999:
                    break
                
                assert adv_para[new_start:].startswith(ans[k]['text']), (adv_para[new_start:], ans[k]['text'])
                ans[k]['answer_start'] = new_start
            
            if die_counter == 9999:
                print('YOU DIED')
                print(adv_para)
                print(orig_para)
                print()
                continue
            
            if title in train_titles:
                if np.random.randint(2, size=1)[0] == 1:
                    train_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, ans, (start, end), 'adv', title)]
            else:
                assert title in test_titles
                if np.random.randint(2, size=1)[0] == 1:
                    test_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, ans, (start, end), 'adv', title)]

YOU DIED
There were two kinds of X.25 networks. There were 7 kinds of X.75 networks. Some such as DATAPAC and TRANSPAC were there originally. Some such as DATAPAC and TRANSPAC were initially implemented with an X.25 external interface. Some older networks such as TELENET and TYMNET were modified to provide a X.25 host interface in addition to older host connection schemes. DATAPAC was developed by Bell Northern Research which was a joint venture of Bell Canada (a common carrier) and Northern Telecom (a telecommunications equipment supplier). Northern Telecom sold several DATAPAC clones to foreign PTTs including the Deutsche Bundespost. X.75 and X.121 allowed the interconnection of national X.25 networks. A user or host could call a host on a foreign network by including the DNIC of the remote network as part of the destination address. [citation needed]
There were two kinds of X.25 networks. Some such as DATAPAC and TRANSPAC were initially implemented with an X.25 external interface. S

In [12]:
np.save('final_train_pairs.npy', train_pairs)
np.save('final_test_pairs.npy', test_pairs)

In [13]:
adv_count = lambda x: sum(map(lambda y: 1 if y[-2] == 'adv' else 0, x))

In [44]:
print('Train:', '%d,' % len(train_pairs), adv_count(train_pairs), 'of which are adversarial')
print('Test:', '%d,' % len(test_pairs), adv_count(test_pairs), 'of which are adversarial')

Train: 1348, 740 of which are adversarial
Test: 886, 494 of which are adversarial


In [15]:
import spacy
nlp = spacy.blank("en")

In [16]:
def word_tokenize(sent):
    doc = nlp(sent)
    return [token.text for token in doc]

In [17]:
def _get_word(word):
    for each in (word, word.lower(), word.capitalize(), word.upper()):
        if each in word2idx_dict:
            return word2idx_dict[each]
    return 1

def _get_char(char):
    if char in char2idx_dict:
        return char2idx_dict[char]
    return 1

In [18]:
with open('data/word2idx.json', 'r') as fh:
    word2idx_dict = json.load(fh)

with open('data/char2idx.json', 'r') as fh:
    char2idx_dict = json.load(fh)

In [19]:
para_limit = 1000
ques_limit = 100
char_limit = 16

In [20]:
import random
random.seed(1337)
random.shuffle(train_pairs)
random.shuffle(test_pairs)

In [21]:
%rm data/combo_v2_*.tf

In [22]:
writer = tf.python_io.TFRecordWriter('data/combo_v2_train.tf')

for id_, (id_but_ignore, context, ques, ans, start_end, tag, topic) in enumerate(train_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    bad_y1 = np.zeros([para_limit], dtype=np.float32)
    bad_y2 = np.zeros([para_limit], dtype=np.float32)
    
    spans = convert_idx(context, context_tokens)
    
    if start_end is None:
        bad_y1[0] = 1
        bad_y2[0] = 1
    else:
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        bad_y1[answer_span[0]] = 1
        bad_y2[answer_span[-1]] = 1
        
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    
    start, end = ans[-1]['answer_start'], ans[-1]['answer_start'] + len(ans[-1]['text'])

    answer_span = []

    for idx, span in enumerate(spans):
        if not (end <= span[0] or start >= span[1]):
            answer_span.append(idx)

    y1[answer_span[0]] = 1
    y2[answer_span[-1]] = 1
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              "bad_y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bad_y1.tostring()])),
                              "bad_y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bad_y2.tostring()])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()

In [23]:
writer = tf.python_io.TFRecordWriter('data/combo_v2_test.tf')
classic_meta_info = {}
meta_info = {}

for id_, (id_but_ignore, context, ques, ans, start_end, tag, topic) in enumerate(test_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    bad_y1 = np.zeros([para_limit], dtype=np.float32)
    bad_y2 = np.zeros([para_limit], dtype=np.float32)
    
    spans = convert_idx(context, context_tokens)
    
    if start_end is None:
        bad_y1[0] = 1
        bad_y2[0] = 1
    else:
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        bad_y1[answer_span[0]] = 1
        bad_y2[answer_span[-1]] = 1
    
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    
    start, end = ans[-1]['answer_start'], ans[-1]['answer_start'] + len(ans[-1]['text'])

    answer_span = []

    for idx, span in enumerate(spans):
        if not (end <= span[0] or start >= span[1]):
            answer_span.append(idx)

    y1[answer_span[0]] = 1
    y2[answer_span[-1]] = 1
    
    meta_info[id_] = {'answers': [a['text'] for a in ans], 'spans': spans, 'context': context, 'uuid': id_but_ignore}
    classic_meta_info[id_] = {'answers': ans, 'spans': spans, 'context': context, 'ques': ques, 'tag': tag, 'title': title, 'uuid': id_but_ignore}
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              "bad_y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bad_y1.tostring()])),
                              "bad_y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bad_y2.tostring()])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()
save('data/combo_v2_test_meta.json', meta_info, message='meta_info')
np.save('data/combo_v2_test_meta.npy', classic_meta_info)

Saving meta_info...


In [24]:
len([a for a in map(lambda x: x['uuid'], classic_meta_info.values())])

1381

In [25]:
len(set([a for a in map(lambda x: x['uuid'], classic_meta_info.values())]))

1381

In [26]:
broken = 0
total = 0

for id_, (id_but_ignore, context, ques, ans, start_end, tag, topic) in enumerate(test_pairs):
    for answer in ans:
        if not context[answer['answer_start']:].startswith(answer['text']):
            broken += 1
        total += 1
        break

print(broken, total)

0 1381


In [28]:
len(set([a for a in map(lambda x: x['uuid'], meta_info.values())]))

1381